# **Indentity Security**  


## Authentication

NVFLARE’s authentication model is based on Public Key Infrastructure (PKI) technology

For the FL project, the Project Admin uses the Provisioning Tool to create a Root CA with a self-signed root certificate. This Root CA will be used to issue all other certs needed by communicating parties.

Identities involved in the project (Server, Clients, participants) are provisioned with the [Provisioning Tool](https://nvflare.readthedocs.io/en/main/real_world_fl/overview.html#provision). Each identity is defined with a unique common name. For each identity, the Provisioning Tool generates a separate password-protected Startup Kit, which includes security credentials for mutual TLS authentication:

* The certificate of the Root CA
* The cert of the identity
* The private key of the identity

Startup Kits are distributed to the intended identities:
* The FL Server’s kit is sent to the Project Admin
* The kit for each FL Client is sent to the Org Admin responsible for the site
* FLARE Console (previously called Admin Client) kits are sent to the user(s)

To ensure the integrity of the Startup Kit, each file in the kit is signed by the Root CA.

Each Startup Kit also contains a “start.sh” file, which can be used to properly start the NVFLARE application.

Once started, the FL client attempts to establish a mutual TLS or standard TLS connection with the FL server, using the PKI credentials in its Startup Kit. This is possible only if both the FL client and the FL server have the correct Startup Kits. 

Similarly, when a user tries to operate the NVFLARE system using the Flare Console app, the Flare Console tries to establish a mutual TLS or standard TLS connection with the Server, using the PKI credentials in its Startup Kits. This is possible only if both the Flare Console and the FL server have the correct Startup Kits. The user must also enter his/her assigned user name correctly.

We supports both normal TLS and mutal TLS. 


The security of the system comes from the PKI credentials in the Startup Kits. As you can see, this mechanism involves manual processing and human interactions for Startup Kit distribution, and hence the identity security of the system depends on the trust of the involved people. To minimize security risk, we recommend that people involved follow these best practice guidelines:

The Project Admin, who is responsible for the provisioning process of the study, should protect the study’s configuration files and store created Startup Kits securely.

When distributing Startup Kits, the Project Admin should use trusted communication methods, and never send passwords of the Startup Kits in the same communication. It is preferred to send the Kits and passwords with different communication methods.

Org Admin and users must protect their Startup Kits and only use them for intended purposes.

>  Note
The provisioning tool tries to use the strongest cryptography suites possible when generating the PKI credentials. All of the certificates are compliant with the X.509 standard. All private keys are generated with a size of 2048-bits. The backend is openssl 1.1.1f, released on March 31, 2020, with no known CVE. All certificates expire within 360 days.

> NVFlare Dashboard is a website that supports user and site registration. Users will be able to download their Startup Kits (and other artifacts) from the website.

Let's see above in action


First, let's generate a default startup kits using the POC (Proof-Of-Concept) command. The POC command internally calls the provision tool to generate the default startup kits

In [ ]:
! echo y | nvflare poc prepare


In [ ]:
! tree /tmp/nvflare/poc/example_project/prod_00

Look at the each site's start up kit. We see

```
startup
│   │   ├── client.crt  <-- certificate
│   │   ├── client.key  <-- private key
│   │   ├── fed_client.json
│   │   ├── rootCA.pem  <-- root certificate
│   │   ├── signature.json
│   │   ├── start.sh
│   │   ├── stop_fl.sh
│   │   └── sub_start.sh

```



## Authorization

Federated learning is conducted over computing resources owned by different organizations. Naturally these organizations have concerns about their computing resources being misused or abused. Even if an NVFLARE docker is trusted by participating orgs, researchers can still bring their own custom code to be part of a study, which could be a big concern to many organizations. In addition, organizations may also have IP (intellectual property) requirements on the studies performed by their own researchers.

Some frameworks disable the BYOC (bring your own code) feature and only allow pre-installed packages during production. However, during research and development, researchers need to modify code constantly during experiments.



NVFLARE comes with an authorization system that can help address these security concerns and IP requirements. With this system, an organization can define strict policy to control access to their computing resources and/or FL jobs.

Here are some examples that an org can do:

* Restrict BYOC to only the org’s own researchers;

* Allow jobs only from its own researchers, or from specified other orgs, or even from specified trusted other researchers;

* Totally disable remote shell commands on its sites

* Allow the “ls” shell command but disable all other remote shell commands


### Centralized vs. Federated Authorization

Instead of relying on a centralized (FL Server) to authorizate and authenticate users, NVFLARE now uses federated authorization where each organization defines and enforces its own authorization policy.

Each organization defines its policy in its own authorization.json (in the local folder of the workspace). This locally defined policy is loaded by FL Clients owned by the organization, The policy is also enforced by these FL Clients.

This decentralized authorization has an added benefit: since each organization takes care of its own authorization, there will be no need to update the policy of any other participants (FL Server or Clients) when new orgs or clients are added.

### Simplified Authorization Policy Configuration
Since each organization defines its own policy, there will be no need to centrally define all orgs and users. The policy configuration for an org is simply a matrix of role/right permissions. Each role/right combination in the permission matrix answers this question: what kind of users in this role can have this right?

To answer this question, the role/right combination defines one or more conditions, and the user must meet one of these conditions to have the right. The set of conditions is called a control.

### Roles
Users are classified into roles. NVFLARE defines four roles:

* **Project Admin** - this role is responsible for the whole FL project;

* **Org Admin** - this role is responsible for the administration of all sites in its org. Each org must have one Org Admin;

* **Lead (researcher)** - this role conducts FL studies

* **Member (researcher)** - this role observes the FL study but cannot submit jobs

### Rights

* Admin commands are grouped into categories. For example, commands like abort_job, delete_job, start_app are in manage_job category; all shell commands are put into the shell_commands category. Each category is also a right.

* BYOC is defined as a right so that some users are allowed to submit jobs with BYOC whereas some are not.

This right system makes it easy to write simple policies that only use command categories. It also makes it possible to write policies to control individual commands. When both categories and commands are used, command-based control takes precedence over category-based control.


## Controls and Conditions


| Notation       | Condition                                           | Examples           |
|----------------|-----------------------------------------------------|--------------------|
| o:site         | The user belongs to the site’s organization         |                    |
| n:submitter    | The user is the job submitter                       |                    |
| o:submitter    | The user and the job submitter belong to the same org|                    |
| n:<person_name>| The user is a specified person                      | n:john@nvidia.com  |
| o:<org_name>   | The user is in a specified org                      | o:nvidia           |

The words “site” and “submitter” are reserved.

For more details please refer [documentation](https://nvflare.readthedocs.io/en/main/user_guide/security/identity_security.html)

Now, lets move on to [site_security](../06.2_site_security_privacy_policy/site_policy.ipynb)
